In [71]:
import pandas as pd

In [72]:
df_MF = pd.read_csv("../data/microsoft_processed.csv")
df_MF.drop_duplicates(subset=("News"), inplace=True)
df_MF = df_MF[df_MF['News'] != " "]
df_MF.rename(columns={'News':'title'},inplace=True)
df_MSFT_unique = pd.read_csv("../data/MSFT_unique_stock_news.csv")
df_MSFT_unique = df_MSFT_unique[df_MSFT_unique["language"] == "English"]
df_MSFT = pd.concat([df_MF,df_MSFT_unique],axis=0,ignore_index=True)
df_MSFT.rename(columns={'Date':'date'},inplace=True)
df_MSFT = df_MSFT[['date','title']]


df_TS = pd.read_csv("../data/tesla_processed.csv")
df_TS.drop_duplicates(subset=("News"), inplace=True)
df_TS = df_TS[df_TS['News'] != " "]
df_TS.rename(columns={'News':'title'},inplace=True)
df_TSLA_trans = pd.read_csv("../data/TSLA_trans_news.csv")
df_TSLA_trans = df_TSLA_trans.drop("title",axis=1)
df_TSLA_trans.rename(columns={'translated':'title'},inplace=True)
df_TSLA_unique = pd.read_csv('../data/TSLA_unique_stock_news.csv')
english_df_TSLA = df_TSLA_unique[df_TSLA_unique['language'] == 'English']
df_TSLA_trans.rename(columns={'translated':'title'},inplace=True)
df_TSLA = pd.concat([df_TS,english_df_TSLA,df_TSLA_trans],axis=0,ignore_index=True)
df_TSLA.rename(columns={'Date':'date'},inplace=True)
df_TSLA = df_TSLA[['date','title']]


df_NVDA = pd.read_csv("../data/NVDA_unique_stock_news.csv",)
df_NVDA = df_NVDA[df_NVDA["language"] == "English"]
df_NVDA.rename(columns={'Date':'date'},inplace=True)
df_NVDA = df_NVDA[['date','title']]
df_NVDA.dropna(inplace=True)

In [73]:
df_MSFT.head()

,date,title
0,2008-02-01 09:30:00,['Microsoft offers to buy Yahoo for $44.6 bn\n...
1,2008-02-01 12:41:00,Microsoft offers to buy Yahoo for $44.6 bn\nTh...
2,2008-02-01 14:50:00,Microsoft 'bear hug' for Yahoo could create Go...
3,2008-02-04 09:30:00,['Microsoft plus Yahoo: Booster dose for MSN i...
4,2008-02-05 09:30:00,['Microsoft may borrow for first time to buy Y...


In [74]:
df_NVDA.head()

,date,title
0,2017-01-15 17:00:00+00:00,Nvidia Shield TV : What The Experts Are Saying
7,2017-01-16 13:45:00+00:00,Nvidia Shield TV ( 2015 ) updated to Android 7...
8,2017-01-16 14:15:00+00:00,Pick Up the New NVIDIA SHIELD TV Today for $199
9,2017-01-16 16:45:00+00:00,NVIDIA SHIELD 2017 Review
10,2017-01-16 18:00:00+00:00,NVIDIA GTX 1080 Ti Revealed By Gigabyte Using ...


In [75]:
df_TSLA.head()

,date,title
0,2015-01-07 09:30:00,['BMW revives wireless charging to reduce elec...
1,2015-01-08 09:30:00,['How oil at $50 could change almost everythin...
2,2015-01-20 09:30:00,['Want to be successful in your 30s? Here are ...
3,2015-01-28 15:00:00,What happens when biggies like Branson and Zuc...
4,2015-01-30 09:30:00,"[""Top Made-in-America cars famous all over the..."


In [76]:
print(df_MSFT.shape,df_TSLA.shape,df_NVDA.shape)

(10685, 2) (7922, 2) (6970, 2)


In [77]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# model_name = "ProsusAI/finbert"
# model_path = r"C:\Users\dalu\Desktop\finbert"   # your real path

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)


# tokenizer.save_pretrained(model_path)
# model.save_pretrained(model_path)


# pipe = pipeline(
#     "sentiment-analysis",
#     model=model,
#     tokenizer=tokenizer,
#     device=0   # GPU if available, else CPU
# )

# print(pipe("Apple stock surged today after earnings."))

In [78]:
import tensorflow as tf
print(tf.__version__)
print(len(tf.config.list_physical_devices("GPU")))
print(tf.config.list_physical_devices("GPU"))

2.8.0
1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [79]:
import torch
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

# ============================
# 1. Load Data
# ============================
def pt_news_sentiment(df):
    start_time = time.perf_counter()
    text_column = "title"
    df[text_column] = df[text_column].fillna("")
    
    # ============================
    # 2. Load Model (safetensors recommended)
    # ============================
    model_name = r"C:\Users\dalu\Desktop\finbert"
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        trust_remote_code=False
    )
    
    # ============================
    # 3. Device
    # ============================
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    print("GPU:", device)
    
    # ============================
    # 4. Parameters
    # ============================
    labels = ["negative", "neutral", "positive"]
    batch_size = 32 if torch.cuda.is_available() else 8
    
    sentiments = []
    scores = []
    
    # ============================
    # 5. Inference Loop
    # ============================
    with torch.no_grad():
        for i in tqdm(range(0, len(df), batch_size), desc="Processing"):
            texts = df[text_column].iloc[i:i+batch_size].tolist()
    
            inputs = tokenizer(
                texts,
                return_tensors="pt",
                truncation=True,
                padding=True,
                max_length=128
            ).to(device)
    
            outputs = model(**inputs)
            probs = torch.softmax(outputs.logits, dim=1)
    
            for p in probs:
                idx = torch.argmax(p).item()
                score = p[idx].item()
                sentiments.append(labels[idx])
                scores.append(score)
    
    # ============================
    # 6. Add Results
    # ============================
    df["sentiment"] = sentiments
    df["score"] = scores
    
    # ============================
    # 7. Save
    # ============================
    # df.to_csv(f"../data/{df}.csv", index=False)
    end_time = time.perf_counter()
        
    elapsed_time = end_time - start_time
    print("-------------")
    print(f'code excuted in: {elapsed_time:.4f} seconds')
    # Add results to DataFrame
    # df["sentiment"] = [r["sentiment"] for r in results]
    # df["score"] = [r["score"] for r in results]
    # print("-------------")
    return df

In [80]:
# df_MSFT_pt_sentiment = pt_news_sentiment(df_MSFT)

In [81]:
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from tqdm import tqdm

# ============================
# 1. Load Data
# ============================
def tf_news_sentiment(df):
    start_time = time.perf_counter()
    text_column = "title"            # change if different
    
    # Clean NaN
    df[text_column] = df[text_column].astype(str)
    df[text_column] = df[text_column].fillna("")
    df = df.reset_index(drop=True)
    
    # ============================
    # 2. Load Model
    # ============================
    model_name = "ProsusAI/finbert"
    model_path = r"C:\Users\dalu\Desktop\finbert"
    tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only = True)
    model = TFAutoModelForSequenceClassification.from_pretrained(
        model_path,
        from_pt=True,
        local_files_only=True# convert from PyTorch
    )
    
    print("GPU available:", tf.config.list_physical_devices('GPU'))
    
    # ============================
    # 3. Parameters
    # ============================
    labels = ["negative", "neutral", "positive"]
    
    # GPU memory safe batch size
    batch_size = 32 if tf.config.list_physical_devices('GPU') else 8
    
    sentiments = []
    scores = []
    
    # ============================
    # 4. Batch Inference Loop
    # ============================
    for i in tqdm(range(0, len(df), batch_size), desc="Processing"):
        batch_texts = df[text_column].iloc[i:i+batch_size].tolist()
    
        inputs = tokenizer(
            batch_texts,
            return_tensors="tf",
            truncation=True,
            padding=True,
            max_length=128
        )
    
        # IMPORTANT: unpack dict
        outputs = model(**inputs)
    
        probs = tf.nn.softmax(outputs.logits, axis=1)
    
        for p in probs:
            idx = int(tf.argmax(p).numpy())
            score = float(tf.reduce_max(p).numpy())
            sentiments.append(labels[idx])
            scores.append(score)
    
    # ============================
    # 5. Add to DataFrame
    # ============================
    df["sentiment"] = sentiments
    df["score"] = scores
    
    # ============================
    # 6. Export
    # ============================
    end_time = time.perf_counter()
        
    elapsed_time = end_time - start_time
    print("-------------")
    print(f'code excuted in: {elapsed_time:.4f} seconds')
    print("-------------")
    return df


In [82]:
df_MSFT_tf_sentiment = tf_news_sentiment(df_MSFT)
df_NVDA_tf_sentiment = tf_news_sentiment(df_NVDA)
df_TSLA_tf_sentiment = tf_news_sentiment(df_TSLA)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Processing: 100%|████████████████████████████████████████████████████████████████████| 334/334 [02:24<00:00,  2.31it/s]


-------------
code excuted in: 152.4835 seconds
-------------


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


...

-------------
code excuted in: 54.9462 seconds
-------------


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Processing: 100%|████████████████████████████████████████████████████████████████████| 248/248 [02:24<00:00,  1.72it/s]

-------------
code excuted in: 145.7589 seconds
-------------


In [95]:
df_MSFT_tf_sentiment['APPL'] = False
df_MSFT_tf_sentiment['MSFT'] = True
df_MSFT_tf_sentiment['NVDA'] = False
df_MSFT_tf_sentiment['TSLA'] = False
df_MSFT_tf_sentiment.head()

,date,title,sentiment,score,MSFT,APPL,NVDA,TSLA
0,2008-02-01 09:30:00,['Microsoft offers to buy Yahoo for $44.6 bn\n...,positive,0.668940,True,False,False,False
1,2008-02-01 12:41:00,Microsoft offers to buy Yahoo for $44.6 bn\nTh...,positive,0.635262,True,False,False,False
2,2008-02-01 14:50:00,Microsoft 'bear hug' for Yahoo could create Go...,positive,0.728866,True,False,False,False
3,2008-02-04 09:30:00,['Microsoft plus Yahoo: Booster dose for MSN i...,positive,0.757488,True,False,False,False
4,2008-02-05 09:30:00,['Microsoft may borrow for first time to buy Y...,positive,0.724632,True,False,False,False


In [96]:
df_NVDA_tf_sentiment['APPL'] = False
df_NVDA_tf_sentiment['MSFT'] = False
df_NVDA_tf_sentiment['NVDA'] = True
df_NVDA_tf_sentiment['TSLA'] = False
df_NVDA_tf_sentiment.head()

,date,title,sentiment,score,NVDA,APPL,MSFT,TSLA
0,2017-01-15 17:00:00+00:00,Nvidia Shield TV : What The Experts Are Saying,positive,0.935091,True,False,False,False
1,2017-01-16 13:45:00+00:00,Nvidia Shield TV ( 2015 ) updated to Android 7...,positive,0.940564,True,False,False,False
2,2017-01-16 14:15:00+00:00,Pick Up the New NVIDIA SHIELD TV Today for $199,positive,0.897323,True,False,False,False
3,2017-01-16 16:45:00+00:00,NVIDIA SHIELD 2017 Review,positive,0.936668,True,False,False,False
4,2017-01-16 18:00:00+00:00,NVIDIA GTX 1080 Ti Revealed By Gigabyte Using ...,positive,0.848646,True,False,False,False


In [97]:
df_TSLA_tf_sentiment['APPL'] = False
df_TSLA_tf_sentiment['MSFT'] = False
df_TSLA_tf_sentiment['NVDA'] = False
df_TSLA_tf_sentiment['TSLA'] = True
df_TSLA_tf_sentiment.head()

,date,title,sentiment,score,TSLA,APPL,MSFT,NVDA
0,2015-01-07 09:30:00,['BMW revives wireless charging to reduce elec...,negative,0.911042,True,False,False,False
1,2015-01-08 09:30:00,['How oil at $50 could change almost everythin...,neutral,0.694870,True,False,False,False
2,2015-01-20 09:30:00,['Want to be successful in your 30s? Here are ...,positive,0.884209,True,False,False,False
3,2015-01-28 15:00:00,What happens when biggies like Branson and Zuc...,positive,0.932450,True,False,False,False
4,2015-01-30 09:30:00,"[""Top Made-in-America cars famous all over the...",positive,0.830828,True,False,False,False


In [98]:
df_news_sentiment = pd.concat([df_MSFT_tf_sentiment,df_TSLA_tf_sentiment,df_NVDA_tf_sentiment],axis=0,ignore_index=True)

In [99]:
df_news_sentiment.head()

,date,title,sentiment,score,MSFT,APPL,NVDA,TSLA
0,2008-02-01 09:30:00,['Microsoft offers to buy Yahoo for $44.6 bn\n...,positive,0.668940,True,False,False,False
1,2008-02-01 12:41:00,Microsoft offers to buy Yahoo for $44.6 bn\nTh...,positive,0.635262,True,False,False,False
2,2008-02-01 14:50:00,Microsoft 'bear hug' for Yahoo could create Go...,positive,0.728866,True,False,False,False
3,2008-02-04 09:30:00,['Microsoft plus Yahoo: Booster dose for MSN i...,positive,0.757488,True,False,False,False
4,2008-02-05 09:30:00,['Microsoft may borrow for first time to buy Y...,positive,0.724632,True,False,False,False


In [100]:
df_news_sentiment.tail()

,date,title,sentiment,score,MSFT,APPL,NVDA,TSLA
25572,2024-12-27 19:45:00+00:00,NVIDIA Unveils The DOOM Mystery Box Under GeF...,positive,0.899294,False,False,True,False
25573,2024-12-28 10:15:00+00:00,Nvidia Just Reached a $30 Billion Milestone . ...,positive,0.887846,False,False,True,False
25574,2024-12-28 10:45:00+00:00,The Nvidia Way,positive,0.902809,False,False,True,False
25575,2024-12-28 19:00:00+00:00,NVIDIA Made Great Strides With Their Open - So...,negative,0.914211,False,False,True,False
25576,2024-12-31 13:30:00+00:00,Nvidia : 180 Billion Reasons Why This Artifici...,positive,0.793314,False,False,True,False


In [101]:
df_news_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25577 entries, 0 to 25576
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       25577 non-null  object 
 1   title      25577 non-null  object 
 2   sentiment  25577 non-null  object 
 3   score      25577 non-null  float64
 4   MSFT       25577 non-null  bool   
 5   APPL       25577 non-null  bool   
 6   NVDA       25577 non-null  bool   
 7   TSLA       25577 non-null  bool   
dtypes: bool(4), float64(1), object(3)
memory usage: 899.3+ KB


In [102]:
df_news_sentiment.to_csv('../data/News_sentiment.csv',index=False)